In [49]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [50]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("HomeSales").getOrCreate()

In [51]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.option('header', 'true').csv(SparkFiles.get('home_sales_revised.csv'), inferSchema=True, sep=',')

# Display S3 bucket as DataFrame
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [52]:
# Import dependencies to change column data types
from pyspark.sql.functions import col, to_date
# Change date_built column data type to date for later query
home_sales_df = home_sales_df.withColumn("date_built", to_date(col("date_built"), "yyyy"))

In [53]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')

In [54]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
"""

query_df = spark.sql(query)
query_df.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [55]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
SELECT YEAR (date_built) AS year_built, ROUND(AVG(price),2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year_built
ORDER BY year_built
"""

query_df = spark.sql(query)
query_df.show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [56]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
SELECT YEAR (date_built) AS year_built, ROUND(AVG(price),2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY year_built
ORDER BY year_built
"""

query_df = spark.sql(query)
query_df.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [57]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

# Begin tracking run time of query
start_time = time.time()

# Establish subquery to calculate the average price
subquery = """
SELECT view, ROUND(AVG(price),2) AS average_price
FROM home_sales
GROUP BY view
ORDER BY view
"""

# Create a temporary view for the subquery
spark.sql(subquery).createOrReplaceTempView('average_prices')

# Main query to filter the results based on average_price
query = """
SELECT view, average_price
FROM average_prices
WHERE average_price < 350000
"""

# Display query results
query_df = spark.sql(query)
query_df.show()

# Print runtime for the query
print("RUN TIME: %.2f seconds" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    285069.21|
|   1|    288248.11|
|   2|    287710.06|
|   3|    284314.53|
|   4|    286307.75|
|   5|    278096.94|
|   6|    284076.65|
|   7|    288929.09|
|   8|    279099.78|
|   9|    282001.65|
|  10|    289693.49|
|  11|    280356.07|
|  12|    286575.22|
|  13|    285849.95|
|  14|    282519.32|
|  15|    284907.04|
|  16|    291990.83|
|  17|    286601.22|
|  18|    287532.36|
|  19|    280347.62|
+----+-------------+
only showing top 20 rows

RUN TIME: 0.39 seconds


In [58]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable('home_sales')

In [59]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [60]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

# Begin tracking run time
start_time = time.time()

# Subquery to calculate the average price
subquery_cache = """
SELECT view, ROUND(AVG(price),2) AS average_price
FROM home_sales
GROUP BY view
ORDER BY view
"""

# Create a temporary view for the subquery
spark.sql(subquery_cache).createOrReplaceTempView('average_prices_cache')

# Query to filter the results based on average_price
query_cache = """
SELECT view, average_price
FROM average_prices_cache
WHERE average_price < 350000
"""

# Display query results
query_cache_df = spark.sql(query_cache)
query_cache_df.show()


# Print runtime for the query
print("RUN TIME: %.2f seconds" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    285069.21|
|   1|    288248.11|
|   2|    287710.06|
|   3|    284314.53|
|   4|    286307.75|
|   5|    278096.94|
|   6|    284076.65|
|   7|    288929.09|
|   8|    279099.78|
|   9|    282001.65|
|  10|    289693.49|
|  11|    280356.07|
|  12|    286575.22|
|  13|    285849.95|
|  14|    282519.32|
|  15|    284907.04|
|  16|    291990.83|
|  17|    286601.22|
|  18|    287532.36|
|  19|    280347.62|
+----+-------------+
only showing top 20 rows

RUN TIME: 0.24 seconds


In [61]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Create parquet file
parquet = "parquet/partitioned_home_sales_data.parquet"

# Partition parquet file by "date_built field"
home_sales_df.write.mode("overwrite").partitionBy("date_built").parquet(parquet)

In [62]:
# 11. Read the parquet formatted data.
partitioned_home_sales_data = spark.read.parquet(parquet)

In [63]:
# 12. Create a temporary table for the parquet data.
partitioned_home_sales_data.createOrReplaceTempView("home_sales_parquet")


In [64]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

#Begin tracking run time
start_time = time.time()

# Subquery to calculate the average price
subquery_parquet = """
SELECT view, ROUND(AVG(price),2) AS average_price
FROM home_sales
GROUP BY view
ORDER BY view
"""
# Create a temporary view for the subquery
spark.sql(subquery_parquet).createOrReplaceTempView('average_prices_parquet')

# Query to filter the results based on average_price
query_parquet = """
SELECT view, average_price
FROM average_prices_parquet
WHERE average_price < 350000
"""
# Display query results
query_parquet_df = spark.sql(query_cache)
query_parquet_df.show()

# Display run time
print("RUN TIME: %.2f seconds" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   0|    285069.21|
|   1|    288248.11|
|   2|    287710.06|
|   3|    284314.53|
|   4|    286307.75|
|   5|    278096.94|
|   6|    284076.65|
|   7|    288929.09|
|   8|    279099.78|
|   9|    282001.65|
|  10|    289693.49|
|  11|    280356.07|
|  12|    286575.22|
|  13|    285849.95|
|  14|    282519.32|
|  15|    284907.04|
|  16|    291990.83|
|  17|    286601.22|
|  18|    287532.36|
|  19|    280347.62|
+----+-------------+
only showing top 20 rows

RUN TIME: 0.42 seconds


In [65]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [66]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False